<a href="https://colab.research.google.com/github/tproffen/ORCSGirlsPython/blob/master/DoodleMining/QuickDrawDemos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://github.com/tproffen/ORCSGirlsPython/blob/master/Images/Logo.png?raw=1" width="10%" align="right" hpsace="50">

# Doodle Data Mining

## Demonstration of using the Quickdraw dataset

* Quickdraw: https://quickdraw.withgoogle.com/ 
* Quickdraw dataset: https://quickdraw.withgoogle.com/data
* Quickdraw API: https://quickdraw.readthedocs.io/en/latest/index.html

* All valid categories are here : https://github.com/googlecreativelab/quickdraw-dataset/blob/master/categories.txt

**More reading**

* https://opensource.googleblog.com/2018/11/introducing-web-component-and-data-api-for-quick-draw.html

## Run these calls first

The two cells below need to be executed first. They install the Quickdraw API and load the needed Python modules.

In [ ]:
!curl -s -o setup.sh https://raw.githubusercontent.com/tproffen/ORCSGirlsPython/master/DoodleMining/Helpers/setup_activity1.sh
!bash setup.sh
#!pip3 install quickdraw

from Helpers.helpers import *

## Simple retrieval of single doodles

Simple retrieval of a doogle and showing it as image.

Let's step it up and -redraw the image with custom background and color and add information about the drawing.

In [ ]:
what="spider"

qd = QuickDrawData()
spider=qd.get_drawing(what)

In [ ]:
spiderimg = spider.get_image(stroke_color=(255, 0, 0), stroke_width=3, bg_color=(255, 255, 255))

plt.figure()
plt.imshow(spiderimg)
plt.axis("off")
plt.show()

print ("Country: ",spider.countrycode)
print ("Date",datetime.fromtimestamp(spider.timestamp))
print ("Number of strokes: ",len(spider.strokes))

## Getting groups of doodles

Now we get a complete group of drawings. These are cached locally. Here we will display the first 10.

In [ ]:
# Loading doodles of pigs (default is 1000)

what="pig"
doodles = QuickDrawDataGroup(what)

In [ ]:
plt.figure(figsize=(20, 15))
# Now we can retrieve doodles from the group

for i in range(15):
  doodle = doodles.get_drawing()
  doodleimg = doodle.get_image(stroke_color=(0, 0, 255), stroke_width=3, bg_color=(255, 220, 220))
  plt.subplot(3, 5, i+1)
  plt.imshow(doodleimg)
  plt.title("Country: "+doodle.countrycode)
  plt.axis('off')

plt.show()


## Look for doodles by country

Since we know the country each doodle came from, we can compare drawing styles :)

In [ ]:
what = "bread"
countries = ["US", "JP", "FR", "CL"]
cols = 7  

all = QuickDrawDataGroup(what, max_drawings=5000, recognized=True)

In [ ]:
!python --version

In [ ]:
plt.figure(figsize=(20, 15))

index=1
for country in countries:
  doodles=all.search_drawings(countrycode=country)
  random.shuffle(doodles)
  print (country,len(doodles))

  for col in range(cols):
    img = doodles[col].get_image(stroke_color=(255,0,0), stroke_width=3, bg_color=(240,240,240))
    plt.subplot(len(countries), cols, index)
    plt.imshow(img)
    plt.title(country)
    plt.axis('off')
    index+=1;

plt.show()

## Let's try an animation of a doodle

In [ ]:
# Get a spider

what="spider"

qd = QuickDrawData()
spider=qd.get_drawing(what)

In [ ]:
# Using the turtle grapics (from Artistic Math) we redraw the doodle

initializeTurtle()
bgcolor('purple')
color('white')
width(5)
showturtle()

for stroke in spider.strokes:
  color(color_random())
  penup()
  for x,y in stroke:
    goto(300+x,200+y)
    pendown()

show()

## Let's draw many of the same doodles

Using transparency, we can see common features.

In [ ]:
# Get a group of doodles

what="circle"

doodles = QuickDrawDataGroup(what, recognized=True, max_drawings=25)

In [ ]:
# Using the turtle grapics (from Artistic Math) we redraw the doodle

initializeTurtle()
bgcolor('white')
color('blue')
width(5)
showturtle()

for d in range(doodles.drawing_count):
  doodle=doodles.get_drawing(d)
  for stroke in doodle.strokes:
    color('rgba(0,0,255,0.02)')
    penup()
    for x,y in stroke:
      goto(300+x,200+y)
      pendown()

show()